In [2]:
import os
import time
import subprocess

import numpy as np
import scipy.signal
import pandas as pd
import scipy.stats

from tvb.simulator.lab import *
from mne import time_frequency, filter
import plotly.graph_objects as go  # for data visualisation
import plotly.io as pio
from toolbox import timeseriesPlot, FFTplot, FFTpeaks, AEC, PLV, PLI, epochingTool, paramSpace

2020-12-18 15:09:38,916 - WARNING - tvb.simulator.common - psutil module not available: no warnings will be issued when a
    simulation may require more memory than available


--- Logging error ---
Traceback (most recent call last):
  File "c:\users\f_r_e\pycharmprojects\brainmodels\venv\lib\site-packages\tvb\simulator\common.py", line 118, in <module>
    import psutil
ModuleNotFoundError: No module named 'psutil'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\F_r_e\AppData\Local\Programs\Python\Python38-32\lib\logging\handlers.py", line 70, in emit
    self.doRollover()
  File "C:\Users\F_r_e\AppData\Local\Programs\Python\Python38-32\lib\logging\handlers.py", line 394, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "C:\Users\F_r_e\AppData\Local\Programs\Python\Python38-32\lib\logging\handlers.py", line 111, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso: 'C:\\Users\\F_r_e\\TVB\\logs\\library.log' -> 'C:\\Users\\F_r_e\\TVB\\logs\\library.log.2020-12-17'
Call stack:


   INFO  log level set to INFO


  File "C:\Users\F_r_e\AppData\Local\Programs\Python\Python38-32\lib\runpy.py", line 193, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\F_r_e\AppData\Local\Programs\Python\Python38-32\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\users\f_r_e\pycharmprojects\brainmodels\venv\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\f_r_e\pycharmprojects\brainmodels\venv\lib\site-packages\traitlets\config\application.py", line 845, in launch_instance
    app.start()
  File "c:\users\f_r_e\pycharmprojects\brainmodels\venv\lib\site-packages\ipykernel\kernelapp.py", line 612, in start
    self.io_loop.start()
  File "c:\users\f_r_e\pycharmprojects\brainmodels\venv\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\F_r_e\AppData\Local\Programs\Python\Python38-32\lib\asyncio\windows_events.py", line 316,

c:\users\f_r_e\pycharmprojects\brainmodels\venv\lib\site-packages\tvb\datatypes\surfaces.py:61: UserWarning: Geodesic distance module is unavailable; some functionality for surfaces will be unavailable.
  warnings.warn(msg)


In [3]:
# This simulation will generate FC for a virtual "Subject".
# Define identifier (i.e. could be 0,1,11,12,...)
subjectid = ".1995JansenRit"
main_folder="C:\\Users\\F_r_e\\PycharmProjects\\brainModels\\"+subjectid
ctb_folder="C:\\Users\\F_r_e\\PycharmProjects\\brainModels\\CTB_data\\output\\"
if subjectid not in os.listdir(ctb_folder):
    os.mkdir(ctb_folder+subjectid)
    os.mkdir(main_folder)

emp_subj = "subj04"

# Prepare bimodality test (i.e. Hartigans' dip test in an external R script via python subprocess
# Build subprocess command: [Rscript, script]
# cmd = ['C:\\Program Files\\R\\R-3.6.1\\bin\\Rscript.exe',
#        'C:\\Users\\F_r_e\\PycharmProjects\\brainModels\\diptest\\diptest.R']

simLength = 4100 # ms - relatively long simulation to be able to check for power distribution
samplingFreq = 1000 #Hz
transient=200 # ms to exclude from timeseries due to initial transient

# integrator: dt=T(ms)=1000/samplingFreq(kHz)=1/samplingFreq(HZ)
# integrator = integrators.HeunStochastic(dt=1000/samplingFreq, noise=noise.Additive(nsig=np.array([5e-6])))
integrator = integrators.HeunDeterministic(dt=1000/samplingFreq)

conn = connectivity.Connectivity.from_file(
    "C:\\Users\\F_r_e\\PycharmProjects\\brainModels\\CTB_data\\output\\CTB_connx66_"+emp_subj+".zip")
conn.weights = conn.scaled_weights(mode="tract")


mon = (monitors.Raw(),)

WARNING  File 'average_orientations' not found in ZIP.
WARNING  File 'cortical' not found in ZIP.
WARNING  File 'hemispheres' not found in ZIP.
WARNING  File 'areas' not found in ZIP.


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\F_r_e\AppData\Local\Programs\Python\Python38-32\lib\logging\handlers.py", line 70, in emit
    self.doRollover()
  File "C:\Users\F_r_e\AppData\Local\Programs\Python\Python38-32\lib\logging\handlers.py", line 394, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "C:\Users\F_r_e\AppData\Local\Programs\Python\Python38-32\lib\logging\handlers.py", line 111, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso: 'C:\\Users\\F_r_e\\TVB\\logs\\library.log' -> 'C:\\Users\\F_r_e\\TVB\\logs\\library.log.2020-12-17'
Call stack:
  File "C:\Users\F_r_e\AppData\Local\Programs\Python\Python38-32\lib\runpy.py", line 193, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\F_r_e\AppData\Local\Programs\Python\Python38-32\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
 

## TVB default parameters

In [1]:
m = models.ReducedWongWang(I_o=np.array([0.33]), J_N=np.array([0.2609]),
                           a=np.array([0.27]), b=np.array([0.108]), d=np.array([154]),
                           gamma=np.array([0.641]), sigma_noise=np.array([0.000000001]),
                           tau_s=np.array([100]), w=np.array([0.6]))

# Coupling function
coup = coupling.Linear(a=np.array([0.0]))

# Run simulation
tic0=time.time()
sim = simulator.Simulator(model=m, connectivity=conn, coupling=coup,  integrator=integrator, monitors=mon)
sim.configure()

output = sim.run(simulation_length=simLength)
print("Simulation time: %0.2f sec" % (time.time() - tic0,))
# Extract data cutting initial transient
raw_data = output[0][1][:, 0, :, 0].T
raw_time = output[0][0][:]
regionLabels = conn.region_labels
regionLabels=list(regionLabels)
regionLabels.insert(0,"AVG")

# average signals to obtain mean signal frequency peak
data = np.asarray([np.average(raw_data, axis=0)])
data = np.concatenate((data, raw_data), axis=0)  # concatenate mean signal: data[0]; with raw_data: data[1:end]

# Check initial transient and cut data
timeseriesPlot(data[0:10], raw_time, regionLabels, main_folder, True)
# Fourier Analysis plot
FFTplot(data[0:10], simLength, regionLabels, main_folder, True)

NameError: name 'models' is not defined

## Parameters from Deco (2013)

In [11]:
m = models.ReducedWongWang(I_o=np.array([0.3]), J_N=np.array([0.2609]),
                           a=np.array([0.27]), b=np.array([0.108]), d=np.array([154]),
                           gamma=np.array([0.641]), sigma_noise=np.array([0.001]),
                           tau_s=np.array([100]), w=np.array([0.9]))

# Coupling function
coup = coupling.Linear(a=np.array([0.0]))

# Run simulation
tic0=time.time()
sim = simulator.Simulator(model=m, connectivity=conn, coupling=coup,  integrator=integrator, monitors=mon)
sim.configure()

output = sim.run(simulation_length=simLength)
print("Simulation time: %0.2f sec" % (time.time() - tic0,))
# Extract data cutting initial transient
raw_data = output[0][1][:, 0, :, 0].T
raw_time = output[0][0][:]
regionLabels = conn.region_labels
regionLabels=list(regionLabels)
regionLabels.insert(0,"AVG")

# average signals to obtain mean signal frequency peak
data = np.asarray([np.average(raw_data, axis=0)])
data = np.concatenate((data, raw_data), axis=0)  # concatenate mean signal: data[0]; with raw_data: data[1:end]

# Check initial transient and cut data
timeseriesPlot(data[0:10], raw_time, regionLabels, main_folder, True)
# Fourier Analysis plot
FFTplot(data[0:10], simLength, regionLabels, main_folder, True)

Simulation time: 3.30 sec
